# Model 1 for Leaf Image Classification Project

### Load Packages and Images

In [ ]:
import numpy as np
import cv2
import os
import pickle
from keras.utils import to_categorical

IMG_ROWS = 200
IMG_COLS = 200
DEPTH = 3
DEFAULT_IMG_SIZE = tuple((IMG_ROWS, IMG_COLS))

DIR = r"C:\Users\chi_b\OneDrive\Desktop\Machine Learning\Leaves Classification\QMST-5367-Machine-Learning-Plant-Image-Processing-main\data"
CATEGORIES = ['poison_ivy', 'raspberry', 'tomato', 'bell_pepper', 'potato']


#LOAD DATA

data = []

def import_data():
    for category in CATEGORIES:
        path = os.path.join(DIR, category)
        num_label = CATEGORIES.index(category)
        
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
                        
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img,DEFAULT_IMG_SIZE)
                data.append([img, num_label])
            except Exception:
                pass
                                     
        
        #save data file:
        data_output= open('leaves_data.pickle', 'wb')
        pickle.dump(data,data_output)
        data_output.close()

import_data()

def load_data():
    data_input = open('leaves_data.pickle', 'rb')
    data = pickle.load(data_input)
    data_input.close()
    
    features = []
    labels = []
    
    for img, label in data:
        features.append(img)
        labels.append(label)
                          
                     
              
        
    features = np.array(features).reshape(-1,IMG_ROWS,IMG_COLS, DEPTH)
        #normalize data
    features = features/255
    features = features.reshape((features.shape[0],IMG_ROWS, IMG_COLS, DEPTH))
    
    labels = np.array(labels)
    labels = to_categorical(labels)
    
    
    return features, labels

### Create and Build Model

In [ ]:
from import_load_data import load_data
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Conv2D,MaxPooling2D, BatchNormalization, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Activation, Flatten, Dense

HEIGHT = 200
WIDTH = 200
DEPTH = 3
VAL_SPLIT = 0.2
BS = 25
LEARNING_RATE = 0.005
EPOCHS = 10
DIR = r"C:\Users\chi_b\OneDrive\Desktop\Machine Learning\Leaves Classification\QMST-5367-Machine-Learning-Plant-Image-Processing-main"

CATEGORIES = ['poison_ivy', 'raspberry', 'tomato', 'bell_pepper', 'potato']
N_CLASSES = len(CATEGORIES)

features, labels = load_data()

#split data
(x_interim, x_test, y_interim, y_test) = train_test_split(features,labels, test_size =0.2)

(x_train, x_val, y_train, y_val) = train_test_split(x_interim, y_interim, test_size =0.2)



#---------------- model building---------------
def model_building():
    checkpoint = ModelCheckpoint('weights.h5', monitor = 'val_loss', save_best_only=True)
    early_stopping_monitor = EarlyStopping(patience=3)
    callbacks_list = [checkpoint, early_stopping_monitor]
    
    #-------------------OPTIMIZE AND REDUCE LOSS-----------------
    print("Optimizing and training network")
    opt = Adam(lr=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
    
       
    model = Sequential()
    

    inputShape = (WIDTH, HEIGHT, DEPTH)

        
    model.add(Conv2D(64, (3, 3), padding="same",input_shape= inputShape, data_format = "channels_last", activation = 'relu'))
    model.add(Conv2D(64, (3, 3), padding="same", activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), padding="same", activation = 'relu'))
    model.add(Conv2D(125, (3, 3), padding="same", activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(256, (3, 3), padding="same", activation = 'relu'))
    model.add(Conv2D(256, (3, 3), padding="same", activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(512, (3, 3), padding="same", activation = 'relu'))
    model.add(Conv2D(512, (3, 3), padding="same", activation = 'relu'))
    model.add(Conv2D(512, (3, 3), padding="same", activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(512, (3, 3), padding="same", activation = 'relu'))
    model.add(Conv2D(512, (3, 3), padding="same", activation = 'relu'))
    model.add(Conv2D(512, (3, 3), padding="same", activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dense(1000, activation = 'relu'))
       
    model.add(Dense(N_CLASSES, activation = "softmax"))

             
    #-----------------Model Summary-----------------
    model.summary()
    
    #compile model
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    
    #image augmentation

    aug = ImageDataGenerator(
        rotation_range=25, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, 
        zoom_range=0.2,horizontal_flip=True, 
        fill_mode="nearest")
    
    history = model.fit_generator(
        aug.flow(x_train, y_train, batch_size=BS),
        validation_data=(x_val, y_val),
        steps_per_epoch=len(x_train) // BS,
        epochs=EPOCHS, 
        verbose=1, callbacks = callbacks_list
        )
     
    return model,history

#--------------------------EVAL MODEL-------------------
def eval_model(model,history):
    import matplotlib.pyplot as plt
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    #Train and validation accuracy
    plt.plot(epochs, acc, 'b', label='Training accurarcy')
    plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
    plt.title('Training and Validation accurarcy')
    plt.legend()
    
    plt.figure()
    #Train and validation loss
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation loss')
    plt.legend()
    plt.show()
    
    print("Calculating model accuracy")
    scores = model.evaluate(x_test, y_test)
    print(f"Test Accuracy: {scores[1]*100}%")

def save_model(model):
    model.save(DIR + r"\project_model.h5")

def load_model():
    import keras
    model = keras.models.load_model(DIR + r"\project_model.h5")
    return model
#-----------------------------------------------------------------

model, history = model_building()

eval_model(model, history)

save_model(model)

